In [ ]:
import re
from tqdm import tqdm
import pickle
from functools import reduce
import mafan
from mafan import text
import itertools
import sys
import os
bos = " <bos> "
eos = " <eos> "

# Tokenizer Functions

## Sentence Tokenizer

In [ ]:
def zng(paragraph):
    for sent in re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U):
        yield sent

## Simplified Chinese Tokenizer

Below is the code for simplified to traditional mapping dictionary.

We have a large dictionary *conversions.txt* that includes words, characters, common phrases, locations and idioms. Each entry contains the traditional chinese word and simplified chinese word.

In [ ]:
infile = open("conversions.txt", "r+", encoding="utf-8")

s2t_dict = dict()

for line in infile:
    line = line.rstrip()
    arr = line.split()
    trad = arr[0]
    sim = arr[1]
    if sim not in s2t_dict:
        s2t_dict[sim] = [trad]
    else:
        s2t_dict[sim].append(trad)
s2t_dict['-'] = ['-']

Tokeniser is used for identifying dictionary words and phrases in the input sentence. We always prefer longer phrases because it gives more meaning and less translation mappings. Hence we use Byte Pair Encoding (BPE) for identifying words, while BPE candidates are constrained by the defined list of vocabs in the dictionary. Since the longest phrase in the dictionary has 8 characters we start with 8-character phrases and do it backwards.

In [ ]:
def tokenizer(sentence, n = 8):
    '''
    This function tokenizes input sentences according to the dicitionary.
    Input: a sentence or paragraph
    Output: a list of tokens from the input in order according to the original paragraph; a list of non-chinese characters from the original text.
    '''
    text, charList = prepare(sentence)
    token_list = []
    input_text = text
    for k in range(n, 0, -1):
        candidates = [input_text[i:i + k] for i in range(len(input_text) - k + 1)]
        for candidate in candidates:
            if candidate in s2t_dict:
                token_list.append(candidate)
                input_text = re.sub(candidate, '', input_text)
    final = sequencer(token_list, text)
    return final, charList

In [ ]:
def output_list(sentence_list, char_list):
    count = 0
    original = [] # sentence we want to output
    
    for word in sentence_list:
        if "-" in word:
            original.append(list(char_list[count]))
            count += 1
        else:
            original.append(word)
    return original

In [ ]:
def output(sentence, char_list):
    count = 0
    original = "" # sentence we want to output

    for char in list(sentence):
        if char == "-":
            original += char_list[count] # append character if non-chinese
            count += 1
        else:
            original += char # append chinese
    return original

In [ ]:
def prepare(sentence):
    new = "" # input to your tokenizer
    char_list = [] # punct / english to be omitted

    for char in list(sentence):
        if text.identify(char) is mafan.NEITHER:
            new += "-" # sub - with non-chinese chars
            char_list.append(char)
        else:
            new += char

    return new, char_list

In [ ]:
def sequencer(tokens, example):

    flags = [1] * len(example)
    sequence = []
    for token in tokens:
        for match in re.finditer(token, example):
            location = (token, match.span()[0], match.span()[1])
            valid = reduce(lambda x,y:x*y, flags[location[1]:location[2]])
            if valid:
                sequence.append(location)
                for i in range(location[1], location[2]):
                    flags[i] = 0
            else:
                continue
    sequence.sort(key=lambda x: x[1])
    result = [x[0] for x in sequence]
    return result

## Corpus Preparation

First, we need to prepare our corpus.
1. We will add paddings (sentinels) to our sentences.
2. Take one sentence at a time.
3. Change non-chinese words to FW to avoid data explosion.
4. Slice the n-grams and add them to dictionary

In [ ]:
def add_stuff(order):
    '''
    This function divides the corpus into n-grams and stores them in dictionary.
    Input: order of n-gram (like 2 for bi-gram)
    Output: none
    '''
    infile = open("hk-zh.txt", "r+") # this contains our corpus
    start_padding = bos * order # add padding
    end_padding = eos * order

    for line in tqdm(infile, total=1314726):
        line = line.rstrip()
        sentences = list(zng(line)) # tokenize sentence by sentence
        for sentence in sentences:
            candidate = start_padding + sentence + end_padding # form sentence
            word_list = candidate.split()
            word_list_tokens = []
            for word in word_list:
                if not(bool(re.match('^[a-zA-Z0-9]+$', word))):
                    word_list_tokens.append(word) # add if not chinese
                else:
                    word_list_tokens.append("FW") # turn non-chinese (except punc) to FW
            word_list = word_list_tokens
            ordered = [word_list[i:i + order] for i in range(1, len(word_list) - order)] # extract n-grams through slicing
            # for each ngram, convert to tuple and add to dictionary
            for ngram in ordered:
                ngram = tuple(ngram)
                if ngram not in corpus:
                    corpus[ngram] = 1
                else:
                    corpus[ngram] += 1

Let's say you want to extract till trigrams.

We want to do 3 iterations, for trigram, bi-gram and then unigram. Each iteration takes 2 minutes. This is only time-consuming part of this code. Once you prep the dictionary, you don't need to do this again.

In [ ]:
corpus = dict()
start_order = 3
for i in range(start_order, 0, -1):
    add_stuff(i)

Once you made the dictionary, dump it into a pickle.

In [ ]:
import pickle
with open('corpus.pkl', 'wb') as handle:
    pickle.dump(corpus, handle)

Here's a way to load a pickle so you don't need to process data everytime.

In [ ]:
with open('corpus.pkl', 'rb') as fp:
    corpus = pickle.load(fp)

# Making Candidate Lists

1. Tokenize the input.
2. Check the mappings of each input.
3. Add all possible mappings to candidate list.

In [ ]:
def convert(sentence):
    '''
    Returns list of possible mappings.
    Input: Simplified chinese sentence
    Output: List of lists. Each list has a set of possible traditional chinese tokens
    '''
    tokens, char_list = tokenizer(sentence)
    candidate_list = []
    for token in tokens:
        candidate_list.append(s2t_dict[token])
    candidate_list = output_list(candidate_list, char_list)
    return(candidate_list)

# Maximum log-likelihood calculations

Compute the log likelihood of a sentence with different \\(\alpha\\) penalties for unigram, bigram and trigrams.

In [ ]:
num_tokens = 4526000 # total number of tokens in corpus

def prob(word_list, alpha_0 = 0.25, alpha_1 = 0.5, alpha_2 = 1.0):
    '''
    Computes the log likelihood probability.
    Input: A sequence of words in form of list
    Output: Log probabilties
    '''
    word_list = tuple(word_list) # change word list to tuple
    if word_list in corpus:
        # word found in dictionary
        numerator = corpus[word_list] # get the frequency of that word list
        denominator = num_tokens # let denominator be num tokens
        # cutoff the last word and check whether it's in corpus
        if len(word_list[:-1]) > 1 and word_list[:-1] in corpus:
            denom_list = word_list[:-1]
            denominator = corpus[denom_list]
        if len(word_list[:-1]) == 1 and word_list[:-1] in corpus:       
            return alpha_0 * log(numerator / denominator) # log of prob*alpha
        elif len(word_list[:-1]) == 2 and word_list[:-1] in corpus:
            return alpha_1 * log(numerator / denominator)
        elif len(word_list[:-1]) == 3 and word_list[:-1] in corpus:
            return alpha_2 * log(numerator / denominator)
        else:
            return log(numerator/denominator)
    else:
        word_list = list(word_list) # convert it back to list
        k = len(word_list) - 1 # backoff, reduce n gram length
        if k > 0:
            # recursive function, divide the sequence into smaller n and find probs
            probs = [prob(word_list[i:i + k]) for i in range(len(word_list) - k + 1)]
            return sum(probs)
        else:
            # we found an unseen word
            if not(bool(re.match('^[a-zA-Z0-9]+$', word_list[0]))):
                return log(1 / num_tokens) # return a small probability
            else:
                return prob(["FW"]) # we encountered a non-chinese word

# Language Model

Generative n-gram language model that estimates the conditional probability of a word given its history in the n-gram. It's calculated by backing off through progressively shorter history models.

Stupid Backoff does not generate normalized probabilities. The main difference is that we don’t apply any discounting and instead directly use the relative frequencies (S is used instead of P to emphasize that these are not probabilities but scores).

\\(S(w^i|w^{i−1}_{i−k+1}) = 
\begin{cases}
    \frac{f(w^{i}_{i−k+1})}{f(w^{i-1}_{i−k+1})} & \text{if } f(w^{i}_{i−k+1})> 0\\
    \\\alpha S(w^i|w^{i−1}_{i−k+2}),              & \text{otherwise}
\end{cases}
\\)

Where \\(\alpha\\) is the backoff factor.

Stupid Backoff is inexpensive to calculate in a distributed environment while approaching the quality
of Kneser-Ney smoothing for large amounts of data.

In [ ]:
from math import log
def backoff(sentence, order, alpha_0 = 0.25, alpha_1 = 0.5, alpha_2 = 1.0):
    '''
    Calcuates log likelihood using backoff language model
    Input: Sentence and order of the n-gram
    Output: Log prob of that sentence
    '''
    score = 0
    sentences = list(zng(sentence)) # sentence tokenizer
    for sentence in sentences:
        start_padding = bos * order # beginning padding
        end_padding = eos * order # ending padding
        candidate = start_padding + sentence + end_padding # add paddings
        word_list = candidate.split()
        word_list_tokens = []
        for word in word_list:
            # append only non-chinese words
            if not(bool(re.match('^[a-zA-Z0-9]+$', word))):
                word_list_tokens.append(word)
            else:
                word_list_tokens.append("FW")
        word_list = word_list_tokens
        ordered = [word_list[i:i + order] for i in range(1, len(word_list) - order)] # shingle into n-grams
        probs = [prob(x, alpha_0, alpha_1, alpha_2) for x in ordered] # calculate probabilities
        score += sum(probs) # final answer
    return score

# Translator

Take simplified sentence as input, generate candidate list for sentence.
For words with many to one mappings add the candidate to a temporary sentence, calculate perplexity and choose the option which gives the lowest perplexity.

Call function to add back spaces at the end and output the final sentence.

In [ ]:
def translate(sentence, alpha_0 = 0.25, alpha_1 = 0.5, alpha_2 = 1.0):
    '''
    Translate a given sentence to traditional
    Input: Simplified Sentence
    Output: Traditional Sentence
    '''
    candidates = convert(sentence) # get the candidate lists
    final_sent = ""
    for words in candidates:
        if len(words) > 1:
            # many to one mappings
            score = -50000.0 # start with extreme negative value
            likely = ""
            for candidate in words:
                temp = final_sent
                temp = temp + " "  + candidate # add a candidate to temp sentence
                current_score = backoff(temp, 3, alpha_0, alpha_1, alpha_2) # check perplexity
                if current_score > score:
                    # if performing good, include that
                    score = current_score
                    likely = candidate
            final_sent = final_sent + " " + likely
        else:
            final_sent = final_sent + " " + words[0]
    final_sent = final_sent.replace(" ", "")
    final_sent = add_back_spaces(sentence, final_sent) #call function to add the spaces back and output translation
    return final_sent

Add spaces back by enumerating through the original and the appended list.

In [ ]:
def add_back_spaces(original, current):
    current_list = list(current)
    original_list = list(original)
    count = 1
    for index, char in enumerate(original_list):
        if char == " ":
            current_list[index - count] += " "
            count += 1
    current = "".join(current_list)
    return current

Test sentence for translate.

In [ ]:
sentence = "早在23岁，伍兹就参与了世界上首个核反应堆Chicago Pile-1的建设，她是导师费米领导的项目团队中最年轻的一员。此外，伍兹在建立和使用实验所需的盖革计数器上起到关键作用。反应堆成功运转并达到自持状态时，她也是唯一在场的女性。曼哈顿计划中，她与费米合作；同时，她曾与第一任丈夫约翰·马歇尔（John Marshall）一同解决了汉福德区钚生产厂氙中毒的问题，并负责监督钚生产反应炉的建造和运行。"
a = translate(sentence)
print(a)

# Joint Probability Based Tokenizer

Greedy tokenizer would generally work for most of the cases, however, it could lead to an undesirable segmentation, due to the preference towards longer chunks.
We propose a joint consideration for sub-word segmentation by considering both source and target sentences.

A translator needs a source sentence $\mathbf{S}$ consisting of segmentations where $\mathbf{S} = s_0 s_1 \dots s_n$ and a target sentence $\mathbf{T}$ consisting of segmentations where $\mathbf{T} = t_0 t_1 \dots t_m$.

We want to find optimal arrangement of $\mathbf{S}$ which is $\mathbf{S}^*$ and optimal arrangement of $\mathbf{T}$ which is $\mathbf{T}^*$. Mathematically:
\begin{align}
\label{eq1}
    \mathbf{S}^*, \mathbf{T}^* = \underset{{s_i \in \mathbf{S}, t_j \in \mathbf{T}}}{\operatorname{argmax}} P(\mathbf{S}, \mathbf{T})
\end{align}
where $P(\mathbf{S}, \mathbf{T})$ is the joint probability of sequences. 

We assume that the prior probabilities, which are $P(\mathbf{S})$ and $P(\mathbf{T})$, are language model based probabilities.

In [ ]:
def dp_tokenizer(sentence):
    s = sentence
    global orig_len
    orig_len = len(s)
    return segment(s)

In [ ]:
model = kenlm.Model("sim_train.klm")

In [ ]:
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [ ]:
def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

We chose Viterbi for segmentation the given sentence. The scoring function is obtained from the constructed language models.

Optimal segmentation depends on the following:
<ol>
    <li>language model score of source sentence of a candidate segment.</li>
    <li>language model score of target sentence of a candidate segment.</li>
    <li>item mapping conversions from source segment to target segment</li>
</ol>

In [ ]:
@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

To avoid OOVs as output segmentations, we imposed a penalty on OOV outputs, which is given by: $\alpha \times \frac{\texttt{len(segment)}}{\texttt{len(sentence)}}$. 

In [ ]:
penalty_constant = 15.0

In [ ]:
def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    sentence = " ".join(words)
    score = 0
    words_ = ['<s>'] + sentence.split() + ['</s>']
    for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
        if oov:
            penalty = len(words_[i+1]) / orig_len
            score += penalty_constant * prob * penalty
        else:
            score += prob
    return score

In [ ]:
import string
alphanumerics = 'a-zA-Z0-9'
known_stops = u'。。…！？'
known_punctuation = u'／（）、，。：「」…。『』！？《》“”；’ ‘【】·〔〕'
eng_punct = string.punctuation
avoid = re.compile("([%s%s%s%s]+)" % (alphanumerics, known_stops, known_punctuation, eng_punct))

# Tokenize Sentence
Tokenize sentence and output the tokens.

In [ ]:
def tokenize_sentence(sentence):
    split_words = re.split(avoid, sentence)
    split_words_values = [(i, bool(re.search(avoid, i))) for i in split_words]
    answer = []
    for (word, value) in split_words_values:
        segmented_text = []
        if value == False:
            orig_len = len(word)
            segmented_text = dp_tokenizer(word)
        else:
            segmented_text = list(word)
        for segs in segmented_text:
            answer.append(segs)
    return answer

In [ ]:
sentence = "姚松炎、周庭势被「DQ」? 泛民质疑，政府再取消参选人资格涉政治筛选，要求律政司司长郑若骅解释法律理据。 有报道指，据全国人大常委会就《基本法》第一百零四条进行的释法，代表泛民参选立法会港岛及九龙西补选的香港众志周庭和被「DQ」前议员姚松炎，势被取消参选资格。律政司表示，法律政策专员黄惠冲将于稍后时间与泛民议员会面，确实时间待定。 民主派议员前晚在律政中心外静坐要求与律政司司长郑若骅会面不果后，昨在立法会召开记者招待会，要求郑就撤销参选人资格的理据，及其给予选举主任的法律意见作出详细交代。公民党议员郭荣铿批评，郑不向公众交代的做法是「冇承担，冇责任」的表现，不能只把责任交托予公务员。"
a = tokenize_sentence(sentence)

# Evaluation

Evaluating the accuracy on 100 lines on sample test data.

In [ ]:
sim_filename = "simplified100.txt"
tra_filename = "traditional100.txt"

In [ ]:
checklist = []
for key in s2t_dict:
    if len(s2t_dict[key]) > 1:
        for t in s2t_dict[key]:
            checklist.append(t)

The translated characters are matched with the original traditional corpus during evaluation. The mismatch characters include wrongly characters and variant characters. Variant characters are characters that are homophones and synonyms. In some cases, simplified characters can have multiple traditional variant characters mapped to them, which gives the same meaning and context. Thus a mismatch in this case does not necessarily means a incorrect conversion. However this is not a common case and does not affect the evaluation result substantially.

In [ ]:
def eval(sim_filename, tra_filename, alpha_0 = 0.25, alpha_1 = 0.5, alpha_2 = 1.0):
    
    total = 0
    correct = 0
    wrong = 0
    micro_total = 0
    micro_correct = 0
    
    sim_file = open(sim_filename, "r+", encoding="utf-8")
    tra_file = open(tra_filename, "r+", encoding="utf-8")
    tra_lines = tra_file.readlines()
    line_count = 0

    for line in sim_file:

        line = line.rstrip()

        line = translate(line, alpha_0 , alpha_1 , alpha_2)
        tra_line = tra_lines[line_count].rstrip()

        if len(line) == len(tra_line):
            char_count = 0
            for c in line:
                total = total + 1
                if c == tra_line[char_count]:
                    correct = correct + 1
                else:
                    # print(c + tra_line[char_count])
                    wrong = wrong + 1

                if tra_line[char_count] in checklist:
                    micro_total += 1
                    if c == tra_line[char_count]:
                        micro_correct = micro_correct + 1

                char_count = char_count + 1

        line_count += 1
    results = []
    results.append(('Total', (total)))
    results.append(('Correct' , (correct)))
    results.append(('Wrong' , (wrong)))
    results.append(('Percentage' , (correct/total*100)))
    results.append(('Micro Total' , (micro_total)))
    results.append(('Micro Correct' , (micro_correct)))
    results.append(('Micro Percentage' , (micro_correct/micro_total*100)))
    
    return results

Overall Accuracy is defined as (no. of correctly converted characters) / (no. of converted characters). We also calculate the Micro-average accuracy to evaluate the performance for one-to-many character conversions only.

In [ ]:
print(eval(sim_filename, tra_filename, 0.7, 0.5, 1))

# Hyper-Parameter Tuning

Testing the evaluation function on the test set with multiple hyperparameter values in order to determine the optimal values for the hyperparameter.

In [ ]:
outfile = open('alpha_tuning.txt', 'w+')
max = [99.5, 0.25, 0.5, 1]
for a_0 in range(100, 10, -10):
    for a_1 in range(100, 10, -10):
        outfile.write(str(eval(sim_filename, tra_filename, alpha_0 = float(a_0)/100.0, alpha_1 = float(a_1)/100.0, alpha_2 = 1.0)) + "a0: " + str(a_0/100.0) + " " + "a1: " + str(a_1/100.0) + '\n')